In [3]:
import os

current_path = os.getcwd()
print(f"Current Working Directory: {current_path}")

Current Working Directory: /Users/txu98/Desktop/CS771/LLM


In [7]:
from model import ModelArgs, Transformer
from tokenizer import Tokenizer
from utils import download_and_unzip
from contextlib import nullcontext
import os
import torch

In [8]:
trained_model_tok32000_id = '1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41'
tok_32000_id = '1UhsXL-ymGFy1fBftMvMbss2PGFzRxZV4'

checkpoint = download_and_unzip(trained_model_tok32000_id, output_dir= "models/trained_model.pt") # e.g., "models/trained_model.pt"
tokenizer = download_and_unzip(tok_32000_id, output_dir="data/trained_tokenizer.model") # e.g., "data/trained_tokenizer.model"

Downloading...
From (original): https://drive.google.com/uc?id=1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41
From (redirected): https://drive.google.com/uc?id=1bJMOyA86CDayzwmU5KjlZnbhCXHUzO41&confirm=t&uuid=d0e093c8-1518-4689-81e6-6d12d7074319
To: /Users/txu98/Desktop/CS771/LLM/models/trained_model.pt/temp.zip
100%|████████████████████████████████████████| 182M/182M [00:04<00:00, 41.4MB/s]


File extracted as: models/trained_model.pt/trained_model_tok32000.pt, saved to models/trained_model.pt


Downloading...
From: https://drive.google.com/uc?id=1UhsXL-ymGFy1fBftMvMbss2PGFzRxZV4
To: /Users/txu98/Desktop/CS771/LLM/data/trained_tokenizer.model/temp.zip
100%|████████████████████████████████████████| 500k/500k [00:00<00:00, 3.09MB/s]

File extracted as: data/trained_tokenizer.model/tok32000.model, saved to data/trained_tokenizer.model


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Use device: {device}")

Use device: cpu


In [11]:
checkpoint_dict = torch.load(checkpoint, map_location=device)
gptconf = ModelArgs(**checkpoint_dict['model_args'])
model = Transformer(gptconf)
state_dict = checkpoint_dict['model']
unwanted_prefix = '_orig_mod.' # sometimes added during compiling
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict, strict=False)

/var/folders/c7/4hz617d51cx2d5h0c8ml6wqw0000gp/T/ipykernel_37815/3284602268.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint, map_

<All keys matched successfully>

In [12]:
model.eval()
model.to(device)

Transformer(
  (tok_embeddings): Embedding(32000, 288)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, ou

In [13]:
enc = Tokenizer(tokenizer_model=tokenizer)

#words: 32000 - BOS ID: 1 - EOS ID: 2


In [14]:
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
torch.backends.cuda.matmul.allow_tf32 = True #enables the use of TF32 for matrix multiplication operations within PyTorch when using CUDA
torch.backends.cudnn.allow_tf32 = True #enables the use of TF32 precision within the cuDNN library
device_type = 'cuda' if 'cuda' in device else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [15]:
num_samples = 1 # number of samples to draw
max_new_tokens = 256 # number of tokens generated in each sample
temperature = 1.0 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 300 # retain only the top_k most likely tokens

In [18]:
start = "Once upon a time, there is a princess called Fangqiao."
start_ids = enc.encode(start, bos=True, eos=False)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(enc.decode(y[0].tolist()))
            print('---------------')

Once upon a time, there is a princess called Fangqiao. He lived in a very tidy and kind heart, ready to help keep the world clear and clean. 
One day, he decided to go for a walk and ventured into his palace. As he listened to the sounds that came from the knights they surrounded him. He became a true guardian, and kept everyone safe thanks to him. 
He stayed with his kingdom and filled his heart with his comfort. He felt so comfortable and inspired by the importance of forgive. He knew that he was lucky to have such a special talent. 
At last, he returned to his palace feeling refreshed. He was pleased with himself, knowing that the compassionate owner had taught him a true moral value, and that truly appreciated is, and that is much more important than boots or cleans. Once upon a time, there was a lady who loved to count. She was very fit and could run and jump all day long. One day, she decided to count the flowers in her garden. She counted each one, one after all. She realized th

In [19]:
   @torch.inference_mode()
   def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
       for _ in range(max_new_tokens):
           idx_cond = idx if idx.size(1) <= self.params.max_seq_len else idx[:, -self.params.max_seq_len:]
           logits = self(idx_cond)
           logits = logits[:, -1, :] # crop to just the final time step
           if temperature == 0.0:
               _, idx_next = torch.topk(logits, k=1, dim=-1)
           else:
               logits = logits / temperature
               if top_k is not None:
                   v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                   logits[logits < v[:, [-1]]] = -float('Inf')
               probs = F.softmax(logits, dim=-1)
               idx_next = torch.multinomial(probs, num_samples=1)
           idx = torch.cat((idx, idx_next), dim=1)

       return idx